In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
test_set = pd.read_csv(r'E:\ThisPC\Desktop\2019 Fall\CS 4661\house-prices-advanced-regression-techniques\test.csv')
train_set = pd.read_csv(r'E:\ThisPC\Desktop\2019 Fall\CS 4661\house-prices-advanced-regression-techniques\train.csv')

In [2]:
# Concat both tables/dataframes except for train_set minus last column/"house price"(label)
data_set1 = pd.concat([test_set,train_set.iloc[:,:-1]])
print(data_set1.head())

#This is experimental, supposedly joining 2 tables like in SQL, in this case doesn't work or is not needed
#data_set2 = pd.merge(test_set, train_set, how='left', left_on = 'Id', right_on = 'Id')
#print(data_set2.head())





     Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0  1461          20       RH         80.0    11622   Pave   NaN      Reg   
1  1462          20       RL         81.0    14267   Pave   NaN      IR1   
2  1463          60       RL         74.0    13830   Pave   NaN      IR1   
3  1464          60       RL         78.0     9978   Pave   NaN      IR1   
4  1465         120       RL         43.0     5005   Pave   NaN      IR1   

  LandContour Utilities  ... ScreenPorch PoolArea PoolQC  Fence MiscFeature  \
0         Lvl    AllPub  ...         120        0    NaN  MnPrv         NaN   
1         Lvl    AllPub  ...           0        0    NaN    NaN        Gar2   
2         Lvl    AllPub  ...           0        0    NaN  MnPrv         NaN   
3         Lvl    AllPub  ...           0        0    NaN    NaN         NaN   
4         HLS    AllPub  ...         144        0    NaN    NaN         NaN   

  MiscVal MoSold  YrSold  SaleType  SaleCondition  
0       0      6

In [3]:
# The test set has 80 columns
print(test_set.head())

     Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0  1461          20       RH         80.0    11622   Pave   NaN      Reg   
1  1462          20       RL         81.0    14267   Pave   NaN      IR1   
2  1463          60       RL         74.0    13830   Pave   NaN      IR1   
3  1464          60       RL         78.0     9978   Pave   NaN      IR1   
4  1465         120       RL         43.0     5005   Pave   NaN      IR1   

  LandContour Utilities  ... ScreenPorch PoolArea PoolQC  Fence MiscFeature  \
0         Lvl    AllPub  ...         120        0    NaN  MnPrv         NaN   
1         Lvl    AllPub  ...           0        0    NaN    NaN        Gar2   
2         Lvl    AllPub  ...           0        0    NaN  MnPrv         NaN   
3         Lvl    AllPub  ...           0        0    NaN    NaN         NaN   
4         HLS    AllPub  ...         144        0    NaN    NaN         NaN   

  MiscVal MoSold  YrSold  SaleType  SaleCondition  
0       0      6

In [4]:
# The train set has 81 columns
print(train_set.head())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [5]:
# This part is only to display new_train_set to hold only columns with numeric values or so deatiled in numerics array
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

new_train_set = train_set.select_dtypes(include=numerics)
print(new_train_set.head())

# We use this set for DecisionTree and LinearRegression
newer_train_set = new_train_set.filter(['MSSubClass','LotArea','OverallQual','OverallCond','YearBuilt','WoodDeckSF','OpenPorchSF','EnclosedPorch','PoolArea','SalePrice'], axis=1)
print(newer_train_set.head())

   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  WoodDeckSF  OpenPorchSF  \
0          2003       196.0         706  ...           0           61   
1          1976         0.0         978  ...         298            0   
2          2002       162.0         486  ...           0           42   
3          1970         0.0         216  ...           0           35   
4          2000       350.0         655  ...         192           84   

   EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold  \
0

In [6]:
# With newer_data_set we have 10 columns including label(SalesPrice)
feature_cols = newer_train_set.columns.values
X=newer_train_set[feature_cols]
y=newer_train_set['SalePrice']
print(feature_cols)
print(X.head())
print(y.head())

['MSSubClass' 'LotArea' 'OverallQual' 'OverallCond' 'YearBuilt'
 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch' 'PoolArea' 'SalePrice']
   MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  WoodDeckSF  \
0          60     8450            7            5       2003           0   
1          20     9600            6            8       1976         298   
2          60    11250            7            5       2001           0   
3          70     9550            7            5       1915           0   
4          60    14260            8            5       2000         192   

   OpenPorchSF  EnclosedPorch  PoolArea  SalePrice  
0           61              0         0     208500  
1            0              0         0     181500  
2           42              0         0     223500  
3           35            272         0     140000  
4           84              0         0     250000  
0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64


In [7]:
#lINEAR REGRESSION only on train set
from sklearn.linear_model import LinearRegression

feature_cols = ['MSSubClass','LotArea','OverallQual','OverallCond','YearBuilt',
 'WoodDeckSF','OpenPorchSF','EnclosedPorch','PoolArea']

X = newer_train_set[feature_cols]
print(X.head())

y = newer_train_set['SalePrice']
print(y.head())

# Splitting the dataset into testing and training:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                    random_state=2)

my_linreg = LinearRegression()
my_linreg.fit(X_train, y_train)

#Checking coefficients
# printing Theta0 using attribute "intercept_":
print(my_linreg.intercept_)
# printing [Theta1, Theta2, Theta3] using attribute "coef_":
print(my_linreg.coef_)

# make predictions on the testing set
y_prediction = my_linreg.predict(X_test)

print(y_prediction)

#RMSE
from sklearn import metrics
import numpy as np

# Calculating "Mean Square Error" (MSE):
mse = metrics.mean_squared_error(y_test, y_prediction)

# Using numpy sqrt function to take the square root and calculate "Root Mean Square Error" (RMSE)
rmse = np.sqrt(mse)

print(rmse) #Following lab_4, I'm getting 57717.2% error x.x

   MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  WoodDeckSF  \
0          60     8450            7            5       2003           0   
1          20     9600            6            8       1976         298   
2          60    11250            7            5       2001           0   
3          70     9550            7            5       1915           0   
4          60    14260            8            5       2000         192   

   OpenPorchSF  EnclosedPorch  PoolArea  
0           61              0         0  
1            0              0         0  
2           42              0         0  
3           35            272         0  
4           84              0         0  
0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64
-854195.2878065024
[-1.48750581e+02  1.42431711e+00  3.62444931e+04  2.51364119e+03
  3.96825694e+02  6.43891437e+01  9.91608915e+01  3.14323512e+01
  3.40611709e+02]
[221608.69954421 184650.22389852 225221.

In [10]:
#Cross-Validation
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.model_selection import cross_val_score

feature_cols = ['MSSubClass','LotArea','OverallQual','OverallCond','YearBuilt',
 'WoodDeckSF','OpenPorchSF','EnclosedPorch','PoolArea']

X = newer_train_set[feature_cols]
y = newer_train_set['SalePrice']

print(X.shape)
print(y.shape)


# Applying 10-fold cross validation with "linear regression":

# In the following line, "my_linreg" is instantiated as an "object" of LinearRegression "class". 
my_linreg = LinearRegression()

mse_list = cross_val_score(my_linreg, X, y, cv=10, scoring='neg_mean_squared_error')

print(mse_list)

### Notice later the rmse.mean is big X.X

# Notice that "cross_val_score" by default provides "negative" values for "mse" to clarify that mse is error.
# in order to calculate root mean square error (rmse), we have to make them positive!
mse_list_positive = -mse_list

# using numpy sqrt function to calculate rmse:
rmse_list = np.sqrt(mse_list_positive)
print(rmse_list)

# calculate the average RMSE as final result of cross validation:
print(rmse_list.mean())

(1460, 9)
(1460,)
[-1.22494589e+09 -1.77270701e+09 -1.85406232e+09 -2.30645589e+09
 -2.91701585e+09 -2.02340711e+09 -1.62037893e+09 -1.58465486e+09
 -3.34063020e+09 -1.42842222e+09]
[34999.22697403 42103.52724312 43058.82394536 48025.57536139
 54009.4051574  44982.29771785 40253.93063753 39807.72357006
 57798.18513138 37794.4734222 ]
44283.31691603262
